In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.7 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO


model = YOLO("../models/yolov8m-pose.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# 코드 돌렸는데.. 파일 용량이 너무 커져서 github 업로드가 불가해져 실행 결과 지웠어요...

from google.colab.patches import cv2_imshow
import cv2

# 비디오 파일을 불러오기 위한 VideoCapture 객체 생성
capture = cv2.VideoCapture("./woman.mp4")
while cv2.waitKey(10) < 0:
    # 현재 프레임이 전체 프레임 수와 같으면 (즉, 영상 끝에 도달하면)
    # 다시 0번 프레임으로 되돌려서 루프 재생
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
    # 비디오로부터 한 프레임 읽기
    ret, frame = capture.read()
    cv2_imshow(frame)
# 자원 해제: 비디오 객체 릴리즈
capture.release()
cv2.destroyAllWindows()

In [ ]:
import torch


def predict(frame, iou=0.7, conf=0.25):
    # 모델에 프레임 입력하여 추론 실행
    # source: 입력 이미지 또는 프레임
    # device: GPU가 있으면 "0", 없으면 CPU 사용
    # iou: Non-Maximum Suppression 시 IOU 임계값
    # conf: 탐지 confidence 임계값
    # verbose=False: 콘솔 출력 억제
    results = model(
        source=frame,
        device="0" if torch.cuda.is_available() else "cpu",
        iou=0.7,
        conf=0.25,
        verbose=False,
    )
     # YOLO는 batch 제출을 가정하므로 results[0]이 첫 번째 결과
    result = results[0]
    return result

In [ ]:
def draw_boxes(result, frame):
  # YOLO 탐지 결과(result.boxes)에 포함된 박스들을 순회
    for boxes in result.boxes:
      # 각 박스 정보: x1, y1, x2, y2, score, class_index
      # tensor -> numpy 로 변환하여 좌표와 점수를 추출
        x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
        # 탐지된 객체의 경계 상자를 원본 프레임에 그림
        # (x1, y1): 좌측 상단, (x2, y2): 우측 하단
        # 색상: 빨강(BGR = 0,0,255), 두께 1
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
    return frame

In [ ]:
import cv2


capture = cv2.VideoCapture("./woman.mp4")
while cv2.waitKey(10) < 0:
    # 현재 프레임이 비디오의 총 프레임 수와 같으면 (즉, 영상 끝에 도달하면)
    # 다시 0번 프레임으로 되돌려서 무한 반복 재생
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    # 프레임을 YOLO 모델에 넣어 객체 탐지 수행
    result = predict(frame)
    # 탐지된 bounding box를 프레임 위에 그림
    frame = draw_boxes(result, frame)
    cv2_imshow(frame)

capture.release()
cv2.destroyAllWindows()

In [ ]:
from ultralytics.yolo.utils.plotting import Annotator

# YOLO pose 결과(result)에서 keypoint를 꺼내어 프레임(frame)에 시각화하는 함수
def draw_keypoints(result, frame):
  # Annotator는 YOLO에서 제공하는 기본 시각화 도구 (스켈레톤 및 포인트를 그림)
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:
        # keypoint tensor 형태 -> (num_keypoints, 3) 형태로 정리
        # squeeze: 차원 제거, cpu 이동 준비
        kps = kps.data.squeeze()
        annotator.kpts(kps)

        nkps = kps.cpu().numpy()
        # nkps[:,2] = 1
        # annotator.kpts(nkps)
        for idx, (x, y, score) in enumerate(nkps):
            if score > 0.5:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), cv2.FILLED)
                cv2.putText(frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)

    return frame

In [ ]:
import cv2


capture = cv2.VideoCapture("./woman.mp4")
while cv2.waitKey(10) < 0:
    # 현재 프레임 번호가 전체 프레임 수와 같으면 (즉, 영상 끝에 도달하면)
    # 다시 0번 프레임으로 돌아가 반복 재생
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    # YOLO 모델을 이용해 객체 탐지 및 pose keypoint 추론 수행
    result = predict(frame)
    frame = draw_boxes(result, frame)
    frame = draw_keypoints(result, frame)
    cv2_imshow(frame)

capture.release()
cv2.destroyAllWindows()